In [1]:
# GOAL: upload data from MPEA_test_data.csv

# import statement format from Lenore's notebook
from citrine import Citrine
import os
import pandas as pd

#Next, import the Dataset Class
from citrine.resources.dataset import Dataset

#Next, import the Data Object Classes
from citrine.resources.measurement_run import MeasurementRun
from citrine.resources.measurement_spec import MeasurementSpec
from citrine.resources.process_run import ProcessRun
from citrine.resources.process_spec import ProcessSpec
from citrine.resources.material_run import MaterialRun
from citrine.resources.material_spec import MaterialSpec
from citrine.resources.ingredient_run import IngredientRun
from citrine.resources.ingredient_spec import IngredientSpec
from citrine.attributes.property import Property
from citrine.attributes.condition import Condition
from citrine.attributes.parameter import Parameter

#Next, import the Template Class
from citrine.resources.process_template import ProcessTemplate
from citrine.resources.measurement_template import MeasurementTemplate
from citrine.resources.material_template import MaterialTemplate
from citrine.resources.property_template import PropertyTemplate
from citrine.resources.condition_template import ConditionTemplate
from citrine.resources.parameter_template import ParameterTemplate

#Finally, Import the Value and Bound Classes
from taurus.entity.bounds.categorical_bounds import CategoricalBounds
from taurus.entity.value.nominal_categorical import NominalCategorical
from taurus.entity.value.nominal_real import NominalReal
from taurus.entity.value.normal_real import NormalReal
from taurus.entity.value.empirical_formula import EmpiricalFormula
from taurus.entity.bounds.real_bounds import RealBounds

In [2]:
API_KEY = os.environ.get('CITRINE_API_KEY')
API_SCHEME = 'https'
API_HOST = 'data-ingestion.citrine-platform.com'
API_PORT = '443'
citrine = Citrine(API_KEY, API_SCHEME, API_HOST, API_PORT)

In [3]:
# initialize project and register a dataset to that project
data_project = citrine.projects.register("MPEA hackathon data upload - final")

dataset_data_info = Dataset(name = "Hackathon test dataset - MPEA data - cborg",
                            summary = "MPEA data for hackathon testing",
                            description = "how are summary and description different?"
                            )

dataset_data = data_project.datasets.register(dataset_data_info)

print("The UID for the new Dataset is: {}".format(dataset_data.uid))

The UID for the new Dataset is: a13fb818-0489-4e1f-8c86-a8429e2e5a82


In [4]:
# Set the template and spec for yield strength

ys_tags = ["template::yield strength::ys"]
ys_property_template_init = PropertyTemplate(name="Yield strength", description = "Template for YS", bounds = RealBounds(0, 10000000, 'MPa'), tags = ys_tags)
ys_property_template = dataset_data.property_templates.register(ys_property_template_init)
print("The UID for the YS template is: {}".format(ys_property_template.uids['id']))

ys_measurement_template_tags = ["measurment::yield strength::ys"]
ys_measurement_template_init = MeasurementTemplate(name = "YS Measurement", properties = [ys_property_template], tags = ys_measurement_template_tags)
ys_measurement_template = dataset_data.measurement_templates.register(ys_measurement_template_init)
print("The UID for the YS measurement template is: {}".format(ys_measurement_template.uids['id']))

ys_spec_tags = ["spec::ys::yield strength"]
ys_spec_init = MeasurementSpec(name = "YS Measurement Spec", template = ys_measurement_template, tags = ys_spec_tags)
ys_spec = dataset_data.measurement_specs.register(ys_spec_init)
print("The UID for the YS Measurement Spec is: {}".format(ys_spec.uids['id']))

The UID for the YS template is: fb4d16e7-e986-4b3f-97d1-5f285ebbbda3
The UID for the YS measurement template is: 5c85938d-fe90-4df4-ab26-d9d8cd987a68
The UID for the YS Measurement Spec is: 99abd456-dc2b-4d0d-b9ad-3f346160f968


In [5]:
# Set the template and spec formula/composition
faux_material_template_tags = ["template::composition"]
faux_material_template_init = MaterialTemplate(name = "Test composition", description = "test comp template", tags = faux_material_template_tags)
faux_material_template = dataset_data.material_templates.register(faux_material_template_init)
print("The UID for the faux material template is: {}".format(faux_material_template.uids['id']))

faux_materials_spec_tags = ["spec::comp::faux material"]
faux_material_spec_init = MaterialSpec(name="Material Spec: test material", template = faux_material_template, tags = faux_materials_spec_tags)
faux_material_spec = dataset_data.material_specs.register(faux_material_spec_init)
print("The UID for the faux material spec is: {}".format(faux_material_spec.uids['id']))



The UID for the faux material template is: e45698a4-ec37-411e-80bc-79af62255e3d
The UID for the faux material spec is: b70b17d7-5dad-4990-817f-3f128fb947bc


In [6]:
# Wrapper function so that each row is a unique material with YS property
def add_ys_measurement(ys_value, composition):
    faux_material_run_init = MaterialRun(name="Material Run: {}".format(composition), spec = faux_material_spec)
    faux_material_run = dataset_data.material_runs.register(faux_material_run_init)
    ys_property = Property(name = "YS", value =  NominalReal(nominal=ys_value, units='MPa'))
    composition_property = Property(name = "Composition", value =  EmpiricalFormula(formula=composition))
    ys_run_tags = ["run::yield strength::ys"]
    ys_run_init = MeasurementRun(name = "YS measurement for {}".format('FORMULA'), spec = ys_spec, material = faux_material_run, properties = [ys_property, composition_property], tags = ys_run_tags)
    ys_run = dataset_data.measurement_runs.register(ys_run_init)
    print("The UID for the YS Measurement Run is: {}".format(ys_run.uids['id']))

In [7]:
# get data and run add_ys_measurement over all rows
df = pd.read_csv('MPEA_test_data.csv')
# df['ys_prop'] = df[' PROPERTY: YS (MPa)'].apply(lambda x: Property(name='YS', value=NominalReal(nominal=x, units='MPa')))

for index, row in df.iterrows():
    add_ys_measurement(row[' PROPERTY: YS (MPa)'], row['FORMULA'])

The UID for the YS Measurement Run is: 93d4b6d7-d6e1-4470-bc2b-c3330b23987e
The UID for the YS Measurement Run is: e1e97e68-14c5-4a3c-96bb-546ea9868ec4
The UID for the YS Measurement Run is: 6760de5d-fa6f-44a4-b29c-482697a6b941
The UID for the YS Measurement Run is: 25a027dd-6755-4532-8285-0fad88ea2df1
The UID for the YS Measurement Run is: 7da00e55-1eb2-496f-be29-af0b99ab6a4f
The UID for the YS Measurement Run is: 7a0d53fd-283e-4039-8489-9f1bae37d669
The UID for the YS Measurement Run is: 6637a5ce-dddf-46cf-8b79-e210c7b38a49
The UID for the YS Measurement Run is: 229fa3b0-4dde-4aed-b6bc-31a0d6a1af56
The UID for the YS Measurement Run is: d36816c3-8b5d-419f-aa74-67869253182c
The UID for the YS Measurement Run is: f9ea0199-e1b0-40dd-b8f3-de7af83cb86b
The UID for the YS Measurement Run is: 521cfe93-d4e3-4557-8fd7-cb6680c544ae
The UID for the YS Measurement Run is: bdfccc4d-e864-4c85-85c6-7e1a41f39063
The UID for the YS Measurement Run is: 39bfa35d-e4e5-49ed-b14b-dc6ddfd14f76
The UID for 